### MNIST classification

In [1]:
require 'nn';
require 'rnn';
matio = require 'matio'

In [2]:
data = matio.load('ex4data1.mat')
trainset = {}
trainset.data = data.X
trainset.label = data.y[{ {}, 1 }]

In [3]:
setmetatable(trainset,
    {__index = function(t,i)
                return {t.data[i], t.label[i]}
        end}
);
 
function trainset:size()
    return self.data:size(1)
end

In [4]:
mean = {}
stdv = {}
for i=1,400 do
    mean[i] = trainset.data[{ {},{i} }]:mean()
    stdv[i] = trainset.data[{ {}, {i} }]:std()
    --print(i .. 'th mean: ' .. mean[i])
    --print(i .. 'th std dev: ' .. stdv[i])
    trainset.data[{ {},{i} }]:add(-mean[i])
    if stdv[i] ~= 0 then
        trainset.data[{ {},{i} }]:div(stdv[i])
    end
end

### define model

In [5]:
batchSize = 5000
rho = 20
hiddenSize = 10
nIndex = 20
nClass = 10

In [6]:
rnn = nn.Sequential()
r = nn.Recurrent(
    hiddenSize, nn.Linear(nIndex, hiddenSize),
    nn.Linear(hiddenSize, hiddenSize), nn.Sigmoid(),
    rho
)
rnn:add(r)
rnn:add(nn.Linear(hiddenSize, nClass))
rnn:add(nn.LogSoftMax())
rnn = nn.Sequencer(rnn)

In [7]:
criterion = nn.SequencerCriterion(nn.ClassNLLCriterion())

### train

In [8]:
lr = 0.1
i = 1

In [9]:
prev = 100
for epoch = 1,1e3 do
    local inputs, targets = {}, {}
    for step=1,rho do
        table.insert(inputs, trainset.data[{{},{(step-1)*rho+1, step*rho}}])
        table.insert(targets, trainset.label[{{}}])
    end

    local outputs = rnn:forward(inputs)
    local err = criterion:forward(outputs, targets)
    if epoch%10 == 1 then print(epoch, err/rho) end
    local gradOutputs = criterion:backward(outputs, targets)
    rnn:backward(inputs, gradOutputs)
    rnn:updateParameters(lr)
    rnn:zeroGradParameters()
    if prev<err and lr > 0.1 then 
        print("prev: ", prev, "cur: ", err, "lr", lr, "->", lr*0.5)
        lr = lr * 0.5 
    end
    prev = err
end

1	2.3341343778711	


11	2.284944357249	


21	2.2713576120618	


31	2.2628512716125	


41	2.2579562236389	

...

961	2.2454906316691	


971	2.1106040505054	


981	2.0651016655989	


991	

2.0501147507605	


### test

In [10]:
correction = {}
for i=10,20 do
    correction[i] = 0
end
for i=1,trainset:size() do
    local answer = trainset.label[i]    
    local inputs = {}
    for step=1,rho do
        table.insert(inputs, trainset.data[{{i},{(step-1)*20+1, step*20}}])
    end
    local prediction = rnn:forward(inputs)
    for d=10,20 do
        guess = prediction[d][{1,{}}]
        local confidences, indices = torch.sort(guess, true)
        -- if i%100 == 1 then print(answer, guess, indices[1]) end
        if (answer == indices[1]) then
            correction[d] = correction[d] + 1
        end
    end
end
for i=10,20 do
    print(i, " = ", correction[i], 100*correction[i]/trainset:size() .. '%')
end

10	 = 	1462	29.24%	
11	 = 	1376	27.52%	
12	 = 	1349	26.98%	
13	 = 	1244	24.88%	
14	 = 	1151	

23.02%	
15	 = 	1044	20.88%	
16	 = 	1140	22.8%	
17	 = 	1250	25%	
18	 = 	1214	24.28%	
19	 = 	1006	20.12%	
20	 = 	

819	16.38%	
